In [1]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-69010315-14df-1d45-0f53-0ab8fed9975a)


In [3]:
import pandas as pd
df = pd.read_csv('/kaggle/input/iam-text/words_new.txt', header = None, sep = " ", on_bad_lines= 'warn')
df.rename(columns={0: "file_name", 8: "text"}, inplace=True)
del df[1], df[2], df[3], df[4], df[5], df[6], df[7]
df = df[df['text'] != 'at']
df = df[df['text'] != ',']
df = df[df['file_name'] != 'a01-117-05-02']
df = df[df['text'] != ')']
#df = df[df['text'] != ' ']
# row_keep = int(len(df) * 0.7)
# df = df.sample(n=row_keep, random_state=30)  # You can change the random_state for different random samples
df.reset_index(drop=True, inplace=True)
len(df)

Skipping line 48: expected 9 fields, saw 10
Skipping line 100: expected 9 fields, saw 10
Skipping line 19778: expected 9 fields, saw 11
Skipping line 19960: expected 9 fields, saw 11
Skipping line 20711: expected 9 fields, saw 11
Skipping line 20743: expected 9 fields, saw 10
Skipping line 20784: expected 9 fields, saw 11
Skipping line 20801: expected 9 fields, saw 11
Skipping line 20822: expected 9 fields, saw 11
Skipping line 20853: expected 9 fields, saw 11
Skipping line 20898: expected 9 fields, saw 11
Skipping line 20930: expected 9 fields, saw 10
Skipping line 20946: expected 9 fields, saw 11
Skipping line 20951: expected 9 fields, saw 10
Skipping line 21001: expected 9 fields, saw 10
Skipping line 21085: expected 9 fields, saw 10
Skipping line 21102: expected 9 fields, saw 11
Skipping line 21162: expected 9 fields, saw 11
Skipping line 21197: expected 9 fields, saw 10



30168

In [4]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # some file names end with jp instead of jpg, the two lines below fix this
        part = file_name.split('-')
        file_name = file_name + '.png'
        file_path = self.root_dir+ '/' + part[0] + '/' + part[0] + '-' + part[1] + '/' + file_name
        # prepare image (i.e. resize + normalize)
        image = Image.open(file_path).convert("RGB")

        try: 
            pixel_values = self.processor(image, return_tensors="pt").pixel_values
            
        except:
            pixel_values = torch.zeros((3, 384, 384))
            #lablels = torch.zeros((128))
            #encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
            #return encoding
            #pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          truncation=True,
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        labels = labels + [-100] * (self.max_target_length - len(labels))

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [5]:
from transformers import DonutProcessor, DonutImageProcessor
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

from transformers import AutoTokenizer, XLMRobertaForCausalLM, AutoConfig
# Load processor
resize = DonutImageProcessor(size={"height": 384, "width": 384})

processor = DonutProcessor(resize, tokenizer)



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [6]:
dataset = IAMDataset(root_dir = '/kaggle/input/iam-handwriting-word-database/iam_words/words', df = df, processor = processor)
len(dataset)

30168

In [7]:
train_size = 0.8
valid_size = 0.2

In [8]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset,
                                    [int(train_size * len(dataset)) + 1,
                                     int(valid_size * len(dataset))])
train_set[0]['labels'].shape

torch.Size([128])

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(valid_set, batch_size=4)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")
model.to(device)

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/809M [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [11]:
encoding = dataset[1]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


Importantly, we need to set a couple of attributes, namely:

the attributes required for creating the decoder_input_ids from the labels (the model will automatically create the decoder_input_ids by shifting the labels one position to the right and prepending the decoder_start_token_id, as well as replacing ids which are -100 by the pad_token_id)
the vocabulary size of the model (for the language modeling head on top of the decoder)
beam-search related parameters which are used when generating text.

In [12]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4


In [13]:
new_emb = model.decoder.resize_token_embeddings(len(processor.tokenizer))
print(f"New embedding size: {new_emb}")
# Adjust our image size and output sequence lengths
model.config.encoder.image_size = (384,384) # (height, width)
model.config.decoder.max_length = 384


New embedding size: Embedding(250002, 1024)


In [14]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
!pip install -q datasets jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [17]:
def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

In [18]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from transformers import AdamW
from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
import wandb


In [21]:
import time

def train(train_dataloader, valid_dataloader, epoch, display_step):
    print(f"Start epoch #{epoch+1}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    train_loss = 0.0

    for i, batch in enumerate(tqdm(train_dataloader)):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)

      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      #train_loss += loss.item()

      #print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

      # Apply gradients
      optimizer.step()

      # Save loss
      train_loss_epoch += loss.item()
      if (i) % display_step == 0:
        print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f}'.format(
                epoch + 1, (i+1) * len(batch[k]), len(train_dataloader.dataset), 100 * (i+1) * len(batch[k]) / len(train_dataloader.dataset),
                loss.item()))

    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch/= (i + 1)

    # Evaluate the validation set
    model.eval()
    valid_cer = 0.0
    with torch.no_grad():
      for batch in tqdm(eval_dataloader):
        # run batch generation
        outputs = model.generate(batch["pixel_values"].to(device))
        # compute metrics
        v_cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
        valid_cer += v_cer
        test_loss_epoch = valid_cer

    print("Validation CER:", valid_cer / len(eval_dataloader))

    test_loss_epoch/= (i+1)

    return train_loss_epoch , test_loss_epoch

In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
wandb.login(
    key = "82599b022f523294b4afeedd55c8e4fcf3c86a48",
)
wandb.init(
    project = "trOCR"
)
epochs = 5
display_step = 431*2

# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0

    (train_loss_epoch, test_loss_epoch) = train(train_dataloader,
                                              eval_dataloader, epoch, display_step)

    if test_loss_epoch < last_loss:
        last_loss = test_loss_epoch

    train_loss_array.append(train_loss_epoch)
    test_loss_array.append(test_loss_epoch)
    wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch})

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vutuanminh-work (deeplearningproj). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231215_173224-p3ndsmxj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lilac-night-34
wandb: ⭐️ View project at https://wandb.ai/deeplearningproj/trOCR
wandb: 🚀 View run at https://wandb.ai/deeplearningproj/trOCR/runs/p3ndsmxj


Start epoch #1


  0%|          | 0/6034 [00:00<?, ?it/s]

Train Epoch: 1 [4/24135 (0.016573441060700227%)]	Loss: 11.6681
Train Epoch: 1 [3452/24135 (14.302879635384297%)]	Loss: 5.0305
Train Epoch: 1 [6900/24135 (28.589185829707894%)]	Loss: 2.7493
Train Epoch: 1 [10348/24135 (42.87549202403149%)]	Loss: 4.3526
Train Epoch: 1 [13796/24135 (57.16179821835509%)]	Loss: 2.6361
Train Epoch: 1 [17244/24135 (71.44810441267869%)]	Loss: 1.1015
Train Epoch: 1 [20692/24135 (85.73441060700227%)]	Loss: 6.5231
Done epoch #1, time for this epoch: 3665.2713203430176s


  0%|          | 0/1509 [00:00<?, ?it/s]

...

Validation CER: 0.6157482796543202
Start epoch #5


  0%|          | 0/6034 [00:00<?, ?it/s]

Train Epoch: 5 [4/24135 (0.016573441060700227%)]	Loss: 1.2055
Train Epoch: 5 [3452/24135 (14.302879635384297%)]	Loss: 1.3180
Train Epoch: 5 [6900/24135 (28.589185829707894%)]	Loss: 2.7974
Train Epoch: 5 [10348/24135 (42.87549202403149%)]	Loss: 0.0554
Train Epoch: 5 [13796/24135 (57.16179821835509%)]	Loss: 0.9786
Train Epoch: 5 [17244/24135 (71.44810441267869%)]	Loss: 0.3540
Train Epoch: 5 [20692/24135 (85.73441060700227%)]	Loss: 0.0040
Done epoch #5, time for this epoch: 3490.7496440410614s


  0%|          | 0/1509 [00:00<?, ?it/s]

Validation CER: 0.5465359490006764


In [24]:
torch.save(model, '/kaggle/working/donut_1e5.pth')